In [1]:
import json
from StringIO import StringIO
import pandas as pd
import pandas as pd
import os
import numpy as np
import datetime
import csv

In [2]:
import findspark
findspark.init()

from pyspark import SparkContext
sc = SparkContext(master="local[4]") 
from pyspark import SparkContext
from pyspark.sql import *
sqlContext = SQLContext(sc)

In [3]:
from pyspark.sql import Row
from pyspark.sql.types import *

In [4]:
listings = pd.read_csv('inside_airbnb/listings.csv')

In [5]:
df_geo = pd.read_csv('park_centers.csv')

In [6]:
schema = StructType([StructField("id", IntegerType(), True),
                    StructField("longitude", DoubleType(), True),
                    StructField("latitude", DoubleType(), True)])

In [7]:
schema_two = StructType([StructField("park_id", IntegerType(), True),
                    StructField("park_longitude", DoubleType(), True),
                    StructField("park_latitude", DoubleType(), True)])

In [8]:
spql_listings_info = sqlContext.createDataFrame(listings[['id', 'longitude', 'latitude']], schema)

In [9]:
spql_parks_info = sqlContext.createDataFrame(df_geo[['park_id', 'center_x', 'center_y']], schema_two)

In [10]:
spql_listings_info_rdd = spql_listings_info.rdd.map(lambda row:(row.id, row.longitude,row.latitude))

In [11]:
spql_parks_info_rdd = spql_parks_info.rdd.map(lambda row:(row.park_id, row.park_longitude,row.park_latitude))

In [12]:
spql_listings_and_parks_info_rdd = spql_listings_info_rdd.cartesian(spql_parks_info_rdd)

In [13]:
spql_listings_and_parks_info_rdd = spql_listings_and_parks_info_rdd.map(lambda x: (x[0][0], x[0][1], x[0][2], x[1][0], x[1][1], x[1][2]))

In [14]:
def distance_calculation(x):
        p_one = float(x[2])
        q_one = float(x[1])
        p_two = float(x[5])
        q_two = float(x[4])
        lon_diff = (q_one - q_two)*np.pi/180
        lat_diff = (p_one - p_two)*np.pi/180
        a = np.sin(lat_diff/2)**2 + np.cos(p_one*np.pi/180)*np.cos(p_two*np.pi/180)*(np.sin(lon_diff/2)**2)
        c = np.arctan2(np.sqrt(a), np.sqrt(1 - a))
        d = 6371.00*float(c)
        return tuple(list(x) + [d])

In [15]:
spql_listings_and_parks_info_rdd = spql_listings_and_parks_info_rdd.map(distance_calculation)

In [16]:
spql_listings_and_parks_info_rdd.take(2)

[(11204286,
  -117.00194036983295,
  32.581881678016465,
  0,
  -117.113,
  32.708,
  8.729150422837035),
 (11204286,
  -117.00194036983295,
  32.581881678016465,
  1,
  -117.161,
  32.708,
  10.228095266922994)]

In [17]:
schema_three = StructType([StructField("id", IntegerType(), True),
                    StructField("longitude", DoubleType(), True),
                    StructField("latitude", DoubleType(), True)] + [StructField("park_id", IntegerType(), True),
                    StructField("park_longitude", DoubleType(), True),
                    StructField("park_latitude", DoubleType(), True), StructField("distance", DoubleType(), True)])

In [18]:
spql_listings_and_parks_info_df = sqlContext.createDataFrame(spql_listings_and_parks_info_rdd, schema_three)

In [19]:
spql_listings_and_parks_info_rdd = spql_listings_and_parks_info_df.rdd.map(lambda row: (row.id, row.longitude, row.latitude, row.park_id, row.park_longitude, row.park_latitude, row.distance))

In [20]:
close_parks_count_sixteen = spql_listings_and_parks_info_rdd.filter(lambda x: x[-1] <= 16).map(lambda x: (x[0], 1)).reduceByKey(lambda x, y : x + y)

In [21]:
closest_park_distance = spql_listings_and_parks_info_rdd.map(lambda x: (x[0], x[-1])).reduceByKey(lambda x, y : min(x, y))

In [22]:
park_distance  = closest_park_distance.map(lambda x: x[1])

In [23]:
park_distance.take(5)

[0.6241338455020371,
 0.4226653619251699,
 0.09824906407752211,
 0.3382018040753475,
 0.2027081563466104]

In [24]:
closest_park_distance = spql_listings_and_parks_info_rdd.cartesian(closest_park_distance).filter(lambda x: x[0][-1] == x[1][-1]).map(lambda x: (x[0][0], x[0][-1], x[0][3]))

In [25]:
close_parks_count_one = spql_listings_and_parks_info_rdd.filter(lambda x: x[-1] <= 1).map(lambda x: (x[0], 1)).reduceByKey(lambda x, y : x + y)
close_parks_count_three = spql_listings_and_parks_info_rdd.filter(lambda x: x[-1] <= 3).map(lambda x: (x[0], 1)).reduceByKey(lambda x, y : x + y)
close_parks_count_five = spql_listings_and_parks_info_rdd.filter(lambda x: x[-1] <= 5).map(lambda x: (x[0], 1)).reduceByKey(lambda x, y : x + y)
close_parks_count_ten = spql_listings_and_parks_info_rdd.filter(lambda x: x[-1] <= 10).map(lambda x: (x[0], 1)).reduceByKey(lambda x, y : x + y)

In [26]:
close_parks_count_three_value = close_parks_count_three.map(lambda x: x[1])
close_parks_count_five_value = close_parks_count_five.map(lambda x: x[1])
close_parks_count_ten_value = close_parks_count_ten.map(lambda x: x[1])
close_parks_count_value = close_parks_count_sixteen.map(lambda x: x[1])

In [27]:
count = 0

In [28]:
import datetime
today = datetime.date.today()
count+=1
filename = 'Datasources/listings_parks/listings_parks_' + str(today) + '_V' + str(count) + '.csv'

In [29]:
print filename

Datasources/listings_parks/listings_parks_2018-05-28_V1.csv


In [30]:
df1 = close_parks_count_one.toDF()

In [31]:
df2 = close_parks_count_three.toDF()

In [32]:
df3 = close_parks_count_five.toDF()

In [33]:
#df4 = close_parks_count_ten.toDF()

In [34]:
df4 = closest_park_distance.toDF()

In [35]:
df1 = df1.toPandas()

In [36]:
df1 = df1.rename(columns={'_1':'listing_id', '_2':'park_count_1km'})

In [37]:
df2 = df2.toPandas()

In [38]:
df2 = df2.rename(columns={'_1':'listing_id', '_2':'park_count_3km'})

In [39]:
df3 = df3.toPandas()

In [40]:
df3 = df3.rename(columns={'_1':'listing_id', '_2':'park_count_5km'})

In [41]:
#df4 = df4.toPandas()

In [ ]:
#df4 = df4.rename(columns={'_1':'listing_id', '_2':'park_count_10km'})

In [ ]:
df4 = df4.toPandas()

In [ ]:
df4 = df4.rename(columns={'_1':'listing_id', '_2':'distance_to_closest_park_km', '_3': 'closest_park_index'})

In [ ]:
t1 = df1.merge(df2, how='inner', on='listing_id')

In [ ]:
t2 = t1.merge(df3, how='inner', on='listing_id')
t3 = t2.merge(df4, how='inner', on='listing_id')

In [ ]:
#t4 = t3.merge(df5, how='inner', on='listing_id')

In [ ]:
t3.head()

In [ ]:
park_data = pd.read_json('Datasources/sdcounty_datasets/parks_datasd.geojson', orient = 'records')

In [ ]:
park_data['park_index'] = range(len(park_data))

In [ ]:
def get_full_park_name(x):
    a = x[u'properties'][u'full_name']
    return a

In [ ]:
def get_park_type(x):
    a = x[u'properties'][u'desig_use']
    return a

In [ ]:
park_data['full_name'] = park_data['features'].map(get_full_park_name)
park_data['park_type'] = park_data['features'].map(get_park_type)

In [ ]:
t3 = t3.merge(park_data[['park_index', 'full_name', 'park_type']], how='outer', left_on='closest_park_index', right_on = 'park_index')

In [ ]:
t3 = t3.dropna()

In [ ]:
t3 = t3.rename(columns = {'full_name': 'closest_park_full_name'})
t3 = t3.drop('park_index', axis = 1)

In [ ]:
t3.to_csv(filename,index=False)

In [ ]:
t3.head()